# LAMMPSのdumpファイルを読み込む

In [1]:
from mlptools.io.read import read_from_lmp_dump

In [2]:
path2lmp_dump = "/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/sputtering/Si/sw/traj_for_ml/0deg_1000eV/dump.lammpstrj"

ase_atoms_list = read_from_lmp_dump(path2lmp_dump, return_ase=True)

Number of frames: 4001


100%|██████████| 4001/4001 [01:12<00:00, 55.08it/s]


In [37]:
# z > 60でスライス, cellも変更する
from mlptools.atoms.extract_atoms import extract_atoms

slices_ase_atoms_list = []
for ase_atoms in ase_atoms_list:
    sliced_ase_atoms = extract_atoms(
        atoms=ase_atoms,
        x_range=[0, ase_atoms.cell[0][0]],
        y_range=[0, ase_atoms.cell[1][1]],
        z_range=[65, ase_atoms.cell[2][2]],
    )
    sliced_ase_atoms.set_chemical_symbols(["Si"] * len(sliced_ase_atoms))
    slices_ase_atoms_list.append(sliced_ase_atoms)

In [41]:
# input.dataに書き込む
from mlptools.io.write import write_from_atoms
from mlptools.io.read import read_from_format
from tqdm import tqdm

path2output = "/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/n2p2/Si/extrapolation_analysis/20231203_sw_sputtering"

with open(f'{path2output}/input.data', mode='a') as f:
    for ase_atoms in tqdm(slices_ase_atoms_list):
        mlpatoms = read_from_format(
            ase_atoms=ase_atoms,
            format="ase",
            has_calculator=False,
            structure_id=None
        )
        try:
            f.write('\n'.join(write_from_atoms(mlpatoms, format='n2p2', structure_id="sw_sputtering")))
        except:
            continue

100%|██████████| 4001/4001 [00:34<00:00, 116.66it/s]
